## Kaggle environment notes and experiments
This notebook contains several experiments performed on Kaggle

## Running library notebooks on Kaggle
TODO: Update this section

### Notebook links
* [00_overview.ipynb](../nbs/00_overview.ipynb)
* [01_explore_layout.ipynb](../nbs/01_explore_layout.ipynb)
* [02_test_layout.ipynb](../nbs/02_test_layout.ipynb)
* [03_explore_rendering.ipynb](../nbs/03_explore_rendering.ipynb)
* [04_explore_plotly_interaction.ipynb](../nbs/04_explore_plotly_interaction.ipynb)
* [05_explore_js_interaction.ipynb](../nbs/05_explore_js_interaction.ipynb)
* [06_test_rendering.ipynb](../nbs/06_test_rendering.ipynb)
* [07_explore_tracing.ipynb](../nbs/07_explore_tracing.ipynb)
* [08_fx_profiling_tutorial.ipynb](../nbs/08_fx_profiling_tutorial.ipynb)
* [09_explore_fx_interpreter.ipynb](../nbs/09_explore_fx_interpreter.ipynb)
* [10_test_tracing.ipynb](../nbs/10_test_tracing.ipynb)
* [11_explore_colors.ipynb](../nbs/11_explore_colors.ipynb)
* [12_test_idlmav.ipynb](../nbs/12_test_idlmav.ipynb)



### Steps
* Open the notebook from the links below
* Paste the following code above the first cell
  ```python
  %pip install git+https://github.com/d112358/idlmav.git
  %pip install torchview
  %pip install torchinfo
  %pip install colorspacious
  ```
* Run the notebook normally



### Notes
* In general, wherever the `plotly_renderer_context` context manager is used, pass `'kaggle'` as its argument for better results on Kaggle

## Information about existing environment

In [ ]:
import sys, os, platform
from datetime import datetime
from pathlib import Path

### OS, Kaggle Kernel and Python

In [ ]:
print(f"Operating System: {platform.system()} {platform.release()} ({platform.version()})")
!cat /etc/os-release
print("")
print(f"Kaggle Kernel Version: {os.getenv('KAGGLE_KERNEL_VERSION')}")
print(f"Python Version: {sys.version}")
print(f"Current Date and Time: {datetime.now()}")

### Existing packages and versions

In [ ]:
import subprocess
filter_strings = ["beautifulsoup", "colorspacious", "fast", "graphviz", "ipy", "jsbeautifier", "matplotlib", "munkres", "nbformat", "numpy", "pandas", "plotly", "tabulate", "timm", "torch"]
result = subprocess.run(["pip", "list"], stdout=subprocess.PIPE, text=True)
lines = result.stdout.split("\n")

filtered_lines = [line for line in lines if any(s in line for s in filter_strings)]
print("\n".join(filtered_lines))

### Memory and disk space

In [ ]:
import psutil, shutil
print(f"Processor: {platform.processor()}")
print(f"CPU Count: {psutil.cpu_count(logical=True)}")
print(f"Total RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")
print(f"Disk Space: {shutil.disk_usage('/').total / 1e9:.2f} GB")

In [ ]:
print("Memory Usage:")
!free -h
print("")
print("Disk Space:")
!df -h

### Internet connectivity

In [ ]:
import requests
try:
    response = requests.get("https://www.google.com", timeout=5)
    print(f"Internet Access: OK (Status Code {response.status_code})")
except requests.ConnectionError:
    print("Internet Access: Failed")

### Git head revision

In [ ]:
!git rev-parse HEAD

## Installation experiments

### Confirm we're running on Kaggle

In [ ]:
import os, warnings

if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
    print('Running in Kaggle')
else:
    warnings.warn('This notebook is designed to be executed in Kaggle. You are not currently running in Kaggle')

class StopExecution(Exception):
    def _render_traceback_(self):
        return []

def check_kaggle():
    if not "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
        print('Skipping cell. This notebook is designed to be executed in Kaggle')
        raise StopExecution

### Install IDLMAV

In [ ]:
check_kaggle()
%pip install git+https://github.com/d112358/idlmav.git

### Additional dependencies used in development and testing
* `timm` and `fastcore` are required for some models with which this library is tested
* `torchview` and `torchinfo` are used to verify the correctness of the model tracing algoritm
  - `graphviz` is required by `torchview`
* `colorspacious` was used to convert RGB colors to CIELAB to select the colors to used for nodes

In [ ]:
check_kaggle()
%pip install timm
%pip install fastcore
%pip install torchview
%pip install torchinfo
%pip install graphviz
%pip install colorspacious

### Install miniai (optional)
* See notes in [setup_vscode_wsl.ipynb](./setup_vscode_wsl.ipynb)
* Some of the notebooks in this library perform experiments on models built with the `miniai` library from the [2022 fastai course](https://course.fast.ai/)

In [ ]:
# Archive the miniai repo to the `tmp` directory and extract the archive to the `miniai` directory
check_kaggle()
from pathlib import Path
Path('tmp').mkdir(exist_ok=True)     # Location to archive the repo to
Path('miniai').mkdir(exist_ok=True)  # Location to extract the archive to
!git -C tmp init -b master
!git -C tmp remote add origin https://github.com/fastai/course22p2.git
!git -C tmp fetch --depth=1 origin master
!git -C tmp archive --format=tar origin/master:miniai | tar -x -C miniai
!rm -rf tmp

In [ ]:
# Install miniai dependencies
check_kaggle()
%pip install fastprogress
%pip install torcheval
%pip install datasets

### Test imports

In [8]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from idlmav import MAV
from miniai.init import clean_mem

## Plotly experiments

In [10]:
import plotly.io as pio
import plotly.graph_objects as go

data = {
    'name': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T'],
    'x': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'y': [10, 15, 13, 17, 11, 13, 18, 14, 12, 19, 17, 15, 10, 19, 18, 13, 12, 17, 14, 15]
}

### Available renderers

In [ ]:
available_renderers = list(pio.renderers)
print(f'Available renderers: {", ".join(available_renderers)}')

### Standard figure

In [ ]:
fig = go.Figure(
    go.Scatter(x=data['x'], y=data['y'], mode='markers',
               hovertemplate=('name: %{customdata}'),
               customdata=data['name']
    )
)
fig.show()


### Figure widget

In [ ]:
widget = go.FigureWidget(
    data=go.Scatter(x=data['x'], y=data['y'], mode='markers',
        hovertemplate=('name: %{customdata}'),
        customdata=data['name']
    )
)

display(fig)

### Figure widget inside ipywidgets container

In [ ]:
import ipywidgets as widgets
widget = go.FigureWidget(
    data=go.Scatter(x=data['x'], y=data['y'], mode='markers',
        hovertemplate=('name: %{customdata}'),
        customdata=data['name']
    )
)

min_y, max_y = min(data['y']), max(data['y'])
slider = widgets.FloatRangeSlider(
    value=[min_y, max_y], min=min_y, max=max_y, step=0.01,
    description='Y Range:', orientation='vertical', continuous_update=True,
    layout=widgets.Layout(height='400px'))

container1 = widgets.HBox([slider, widget])

def update_plot(change):
    widget.update_layout(yaxis=dict(range=slider.value))

slider.observe(update_plot, names="value")
display(container1)

## IDLMAV experiments

In [ ]:
import torch, torchvision
from idlmav import MAV, plotly_renderer_context
from IPython.display import display
device = 'cpu'
model = torchvision.models.resnet18().to(device)
x = torch.randn(16,3,160,160).to(device)
mav = MAV(model, x, device=device)
container2 = mav.draw_interactive_graph(add_slider=True, add_overview=False, add_table=False)
display(container2)